In [1]:
import pandas as pd
import numpy as np
import os, sys
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
f,a = plt.subplots(1,1,figsize=(1e-2, 1e-2))
mpl.rcParams['figure.dpi'] = 180
sns.set_style('darkgrid')
import os,sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings("ignore")

In [2]:
from src.mutation_tools import pipeline_mutation_scores

In [3]:
d = '../data/'
list(filter(lambda x: 'pepx' in x.lower() or 'aligned' in x.lower(), os.listdir(d)))

['pepx', '230829_sine_processed_pepx_mutations.csv', 'aligned_icore']

In [4]:
%ls {d}aligned_icore/

230125_cedar_aligned_icore.csv   230308_prime_aligned_icore.csv
230125_ibel_aligned_icore.csv    230418_cedar_aligned.csv
230125_nepdb_aligned_icore.csv   230418_nepdb_aligned.csv
230125_prime_aligned_icore.csv   230418_prime_aligned.csv
230308_cedar_aligned_icore.csv   230717_cedar_aligned_icore.csv
230308_ibel_aligned_icore.csv    backup/
230308_nepdb_aligned_icore.csv   backup_230308_beforeForeignness/


In [5]:
%ls {d}pepx/

230125_cedar_aligned_pepx_fold.csv      230308_prime_aligned_pepx.csv
230125_cedar_related_pepx_fold.csv      230418_cedar_aligned_pepx.csv
230125_ibel_aligned_pepx.csv            230418_nepdb_aligned_pepx.csv
230125_ibel_normal_pepx.csv             230418_prime_aligned_pepx.csv
230125_nepdb_aligned_pepx.csv           2308XX_cedar_wildtype.csv
230125_nepdb_normal_pepx.csv            2308XX_cedar_wildtype_pepx_output.csv
230125_prime_aligned_pepx.csv           2308XX_cedar_wt_icores.csv
230125_prime_related_pepx.csv           2308XX_cedar_wt_icores_pepx_output.csv
230308_cedar_aligned_pepx_fold.csv      TCGA_PANCAN/
230308_ibel_aligned_pepx.csv            backup/
230308_nepdb_aligned_pepx.csv


## do for aligned icores

In [35]:
# aligned icores first
cedar = pd.read_csv(f'{d}aligned_icore/230418_cedar_aligned.csv')
prime = pd.read_csv(f'{d}aligned_icore/230418_prime_aligned.csv')
nepdb = pd.read_csv(f'{d}aligned_icore/230418_nepdb_aligned.csv')

to_drop = [x for x in cedar.columns if 'unnamed' in x.lower() or 'vs' in x.lower() or 'improve' in x.lower() or 'mixed_' in x.lower() or '_input' in x.lower()]

In [36]:
from src.utils import pkl_load
ics_kl = pkl_load('../data/ic_dicts/ics_kl_new.pkl')
cedar = pipeline_mutation_scores(cedar, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')
prime = pipeline_mutation_scores(prime, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')
nepdb = pipeline_mutation_scores(nepdb, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')

In [37]:
cedar.drop(columns=to_drop).to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_cedar_aligned_trainData.csv', index=False)
prime.query('Peptide not in @cedar.Peptide.values')\
     .drop(columns=[x for x in to_drop if x in prime.columns])\
     .to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_prime_aligned_testData.csv', index=False)
nepdb.query('Peptide not in @cedar.Peptide.values')\
     .drop(columns=[x for x in to_drop if x in nepdb.columns])\
     .to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_nepdb_aligned_testData.csv', index=False)

In [38]:
prime.query('Peptide not in @cedar.Peptide.values').drop(columns=[x for x in to_drop if x in prime.columns])

,HLA,agg_label,fold,Peptide,wild_type,icore_mut,core_mut,len,EL_rank_mut,EL_rank_wt,...,isoelectric_point,VHSE1,VHSE3,VHSE7,VHSE8,seq_id,core_wt_aligned,EL_rank_wt_aligned,ratio_rank,foreignness_score
0,HLA-B0702,0,0,AAAAAAAAVL,AAAAAAAAVI,AAAAAAAVL,AAAAAAAVL,10,0.9883,2.6958,...,6.100000,0.352222,-1.028889,0.262222,-0.445556,>seq_1,AAAAAAAVI,3.9965,0.743902,0.000000e+00
1,HLA-B0702,0,0,AAAAAAAVL,AAAAAAAVI,AAAAAAAVL,AAAAAAAVL,9,0.9883,3.9965,...,6.100000,0.352222,-1.028889,0.262222,-0.445556,>seq_2,AAAAAAAVI,3.9965,0.743902,0.000000e+00
2,HLA-A1101,0,5,AAAALHMQR,AAAALHMLR,AAAALHMQR,AAAALHMQR,9,0.3609,0.3717,...,10.551759,0.012222,-0.313333,0.298889,-0.238889,>seq_3,AAAALHMLR,0.3717,0.010486,0.000000e+00
3,HLA-A0301,0,0,AAAKAKNIPK,AAAKAKFIPK,AAKAKNIPK,AAKAKNIPK,10,0.6000,0.3640,...,11.103071,-0.284444,-0.280000,0.705556,0.175556,>seq_4,AAKAKFIPK,0.3640,0.389011,3.475804e-09
4,HLA-B0702,0,2,AAAPAVGPPL,AAATAVGPPL,APAVGPPL,APA-VGPPL,10,0.0897,1.1108,...,6.100000,0.360000,-0.830000,0.250000,0.966250,>seq_5,ATA-VGPPL,4.7794,0.088017,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3104,HLA-A2402,0,8,YYCEVLPGEI,YYREVLPGEI,YYCEVLPGEI,YYCEVLPEI,10,0.9916,1.2384,...,3.613682,0.245000,-0.056000,-0.021000,0.025000,>seq_3105,YYREVLPEI,1.2384,0.197615,0.000000e+00
3105,HLA-A2402,0,9,YYCHWGYEV,YYCHGGYEV,YYCHWGYEV,YYCHWGYEV,9,0.4481,0.4865,...,5.360895,0.273333,0.278889,-0.210000,-0.366667,>seq_3106,YYCHGGYEV,0.4865,0.035369,0.000000e+00
3106,HLA-A2402,0,1,YYEDIKNDF,YYEDIKKDF,YYEDIKNDF,YYEDIKNDF,9,0.0940,0.1546,...,3.876012,-0.181111,0.356667,-0.008889,-0.100000,>seq_3107,YYEDIKKDF,0.1546,0.036846,2.668819e-11
3107,HLA-A2402,0,9,YYNCAFTRNI,YYNRAFTRNI,YYNCAFTRNI,YYNAFTRNI,10,1.3054,0.7245,...,8.506546,0.055000,0.174000,-0.269000,-0.242000,>seq_3108,YYNRAFTRI,0.7245,1.046662,7.619771e-03


## redo for pepx

In [39]:
# aligned icores first
cedar = pd.read_csv(f'{d}pepx/230418_cedar_aligned_pepx.csv')
prime = pd.read_csv(f'{d}pepx/230418_prime_aligned_pepx.csv')
nepdb = pd.read_csv(f'{d}pepx/230418_nepdb_aligned_pepx.csv')

to_drop = [x for x in cedar.columns if 'unnamed' in x.lower() or 'vs' in x.lower() or 'improve' in x.lower() or 'mixed_' in x.lower() or '_input' in x.lower()]

In [40]:
from src.utils import pkl_load
ics_kl = pkl_load('../data/ic_dicts/ics_kl_new.pkl')
cedar = pipeline_mutation_scores(cedar, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')
prime = pipeline_mutation_scores(prime, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')
nepdb = pipeline_mutation_scores(nepdb, 'icore_mut', 'icore_wt_aligned', ics_kl, 0.2, 'icore_')

In [41]:
cedar.drop(columns=to_drop).to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_cedar_expression_trainData.csv', index=False)
prime.query('Peptide not in @cedar.Peptide.values')\
     .drop(columns=[x for x in to_drop if x in prime.columns])\
     .to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_prime_expression_testData.csv', index=False)
nepdb.query('Peptide not in @cedar.Peptide.values')\
     .drop(columns=[x for x in to_drop if x in nepdb.columns])\
     .to_csv('../../../writing/230131_cedar_paper_1/submission_nar/2304XX_nepdb_expression_testData.csv', index=False)

In [42]:
prime.query('Peptide not in @cedar.Peptide.values').drop(columns=[x for x in to_drop if x in prime.columns])

,HLA,agg_label,fold,Peptide,wild_type,icore_mut,core_mut,len,EL_rank_mut,EL_rank_wt,...,seq_id,core_wt_aligned,EL_rank_wt_aligned,Total_Peptide_TPM,Total_Scaled_Peptide_TPM,Total_Gene_TPM,foreignness_score,ratio_rank,ic_len,ratio_rank_squared
0,HLA-B0702,0,0,AAAAAAAAVL,AAAAAAAAVI,AAAAAAAVL,AAAAAAAVL,10,0.9883,2.6958,...,>seq_1,AAAAAAAVI,3.9965,26.261,8.165,26.261400,0.000000e+00,0.743902,73978,0.244398
1,HLA-B0702,0,0,AAAAAAAVL,AAAAAAAVI,AAAAAAAVL,AAAAAAAVL,9,0.9883,3.9965,...,>seq_2,AAAAAAAVI,3.9965,26.261,8.165,26.261400,0.000000e+00,0.743902,73978,0.244398
2,HLA-A1101,0,5,AAAALHMQR,AAAALHMLR,AAAALHMQR,AAAALHMQR,9,0.3609,0.3717,...,>seq_3,AAAALHMLR,0.3717,5.331,1.066,5.330980,0.000000e+00,0.010486,74135,0.350414
3,HLA-A0301,0,0,AAAKAKNIPK,AAAKAKFIPK,AAKAKNIPK,AAKAKNIPK,10,0.6000,0.3640,...,>seq_4,AAKAKFIPK,0.3640,7.751,7.751,7.751090,3.475804e-09,0.389011,61640,0.989011
4,HLA-B0702,0,2,AAAPAVGPPL,AAATAVGPPL,APAVGPPL,APA-VGPPL,10,0.0897,1.1108,...,>seq_5,ATA-VGPPL,4.7794,0.631,0.631,0.630995,0.000000e+00,0.088017,17862,0.001683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,HLA-A2402,0,8,YYCEVLPGEI,YYREVLPGEI,YYCEVLPGEI,YYCEVLPEI,10,0.9916,1.2384,...,>seq_3105,YYREVLPEI,1.2384,5.821,1.938,5.821130,0.000000e+00,0.197615,21635,0.793985
2988,HLA-A2402,0,9,YYCHWGYEV,YYCHGGYEV,YYCHWGYEV,YYCHWGYEV,9,0.4481,0.4865,...,>seq_3106,YYCHGGYEV,0.4865,0.261,0.196,0.260996,0.000000e+00,0.035369,52592,0.412731
2989,HLA-A2402,0,1,YYEDIKNDF,YYEDIKKDF,YYEDIKNDF,YYEDIKNDF,9,0.0940,0.1546,...,>seq_3107,YYEDIKKDF,0.1546,7.211,4.204,7.211000,2.668819e-11,0.036846,52592,0.057154
2990,HLA-A2402,0,9,YYNCAFTRNI,YYNRAFTRNI,YYNCAFTRNI,YYNAFTRNI,10,1.3054,0.7245,...,>seq_3108,YYNRAFTRI,0.7245,9.631,5.779,9.631100,7.619771e-03,1.046662,21635,2.352062
